In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from statsmodels.tools.eval_measures import mse, rmse
from sqlalchemy import create_engine
from sklearn.metrics import mean_absolute_error
import seaborn as sns
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV

import warnings
warnings.filterwarnings('ignore')

nyc_df = pd.read_csv('nyc_census_tracts.csv')

In [2]:
nyc_df.head()

,CensusTract,County,Borough,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,36005000100,Bronx,Bronx,7703,7133,570,29.9,6.1,60.9,0.2,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,36005000200,Bronx,Bronx,5403,2659,2744,75.8,2.3,16.0,0.0,...,2.9,0.0,0.0,43.0,2308,80.8,16.2,2.9,0.0,7.7
2,36005000400,Bronx,Bronx,5915,2896,3019,62.7,3.6,30.7,0.0,...,1.4,0.5,2.1,45.0,2675,71.7,25.3,2.5,0.6,9.5
3,36005001600,Bronx,Bronx,5879,2558,3321,65.1,1.6,32.4,0.0,...,8.6,1.6,1.7,38.8,2120,75.0,21.3,3.8,0.0,8.7
4,36005001900,Bronx,Bronx,2591,1206,1385,55.4,9.0,29.0,0.0,...,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2


In [3]:
nyc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2167 entries, 0 to 2166
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CensusTract      2167 non-null   int64  
 1   County           2167 non-null   object 
 2   Borough          2167 non-null   object 
 3   TotalPop         2167 non-null   int64  
 4   Men              2167 non-null   int64  
 5   Women            2167 non-null   int64  
 6   Hispanic         2128 non-null   float64
 7   White            2128 non-null   float64
 8   Black            2128 non-null   float64
 9   Native           2128 non-null   float64
 10  Asian            2128 non-null   float64
 11  Citizen          2167 non-null   int64  
 12  Income           2101 non-null   float64
 13  IncomeErr        2101 non-null   float64
 14  IncomePerCap     2121 non-null   float64
 15  IncomePerCapErr  2121 non-null   float64
 16  Poverty          2125 non-null   float64
 17  ChildPoverty  

In [4]:
nyc_df = nyc_df.dropna()

In [5]:
nyc_df.isnull().sum()

CensusTract        0
County             0
Borough            0
TotalPop           0
Men                0
Women              0
Hispanic           0
White              0
Black              0
Native             0
Asian              0
Citizen            0
Income             0
IncomeErr          0
IncomePerCap       0
IncomePerCapErr    0
Poverty            0
ChildPoverty       0
Professional       0
Service            0
Office             0
Construction       0
Production         0
Drive              0
Carpool            0
Transit            0
Walk               0
OtherTransp        0
WorkAtHome         0
MeanCommute        0
Employed           0
PrivateWork        0
PublicWork         0
SelfEmployed       0
FamilyWork         0
Unemployment       0
dtype: int64

In [6]:
nyc_df = nyc_df.drop(columns = ['IncomePerCapErr','IncomeErr'])

In [7]:
# Convert boroughs to dummies
nyc_df = pd.get_dummies(nyc_df)
nyc_df.head()

,CensusTract,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Citizen,...,County_Bronx,County_Kings,County_New York,County_Queens,County_Richmond,Borough_Bronx,Borough_Brooklyn,Borough_Manhattan,Borough_Queens,Borough_Staten Island
1,36005000200,5403,2659,2744,75.8,2.3,16.0,0.0,4.2,3639,...,1,0,0,0,0,1,0,0,0,0
2,36005000400,5915,2896,3019,62.7,3.6,30.7,0.0,0.3,4100,...,1,0,0,0,0,1,0,0,0,0
3,36005001600,5879,2558,3321,65.1,1.6,32.4,0.0,0.0,3536,...,1,0,0,0,0,1,0,0,0,0
4,36005001900,2591,1206,1385,55.4,9.0,29.0,0.0,2.1,1557,...,1,0,0,0,0,1,0,0,0,0
5,36005002000,8516,3301,5215,61.1,1.6,31.1,0.3,3.3,5436,...,1,0,0,0,0,1,0,0,0,0


In [8]:
nyc_df['Pov_Unemploy'] = nyc_df['Poverty'] * nyc_df['Unemployment']

In [9]:
#Test and Training Split
import statsmodels.api as sm
X = nyc_df[['Pov_Unemploy','Professional','White','SelfEmployed','Service','Borough_Queens','Production']]
Y = nyc_df['Income']
# We need to manually add a constant
# in statsmodels' sm
X = sm.add_constant(X)
#split the data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)



In [10]:
# Let's try a Random Forest model.
from sklearn.ensemble import RandomForestRegressor

estimator = RandomForestRegressor()

param_grid = { 'max_depth':range(3,7),'n_estimators':[100,200,300,400,500],
                'bootstrap':[True,False]}

forest_grid = GridSearchCV(estimator, param_grid, cv=5, scoring = 'neg_mean_squared_error',verbose=0,n_jobs=-1)
forest_grid.fit(X_train, y_train)
y_pred = forest_grid.predict(X_test)

from sklearn import metrics

print('Mean Absolute Error of the prediction:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Income in Dataset:', np.mean(nyc_df['Income']))
print('The Random Forest Regressor is predicting the median household income at {}%'.format(100*(1 - (metrics.mean_absolute_error(y_test, y_pred))/np.mean(nyc_df['Income']))))
print('Root Mean Squared Error of the prediction:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
Forest_RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print(RandomForestRegressor.score)

Mean Absolute Error of the prediction: 9821.005011470133
Mean Income in Dataset: 59135.96372315036
The Random Forest Regressor is predicting the median household income at 83.39250027707685%
Root Mean Squared Error of the prediction: 13664.630730804749
<function RegressorMixin.score at 0x191acd06a8>


In [11]:
# Try the MLP Model on the Data.

# Import the model.
from sklearn.neural_network import MLPRegressor
# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPRegressor(hidden_layer_sizes=(1000),activation='relu')
mlp.fit(X_train, y_train)
y_predmlp = mlp.predict(X_test)


In [12]:
print(y_predmlp)

[ 47790.39999448  38108.24235593  30066.80448414  36613.69923323
  44586.94399741  33868.77500565  95216.53548932  39012.75410464
  72460.28203346  66256.68240514  48917.78275739  67804.57805448
  25245.57396388  69708.10191659  38595.81233152  42154.54134345
  76057.40437058  64543.8584054   25736.93061699  29557.38393835
  39007.32714304  79353.12145705  98708.3075872   48007.82650677
  46032.413134    30288.39664662  72138.53722536  46593.44084105
  35889.93600118  26255.05205972  33932.51385324  48503.76071113
  33765.25160993  27420.32102392  36168.55112728  50615.83468495
  26006.05376364  24970.14888757  38536.61290404  29141.37781903
  37745.98265699  63406.10325676  72858.68958767  33667.28115031
  95553.6914432   36058.73583543  49590.6745617   44495.87923696
  46259.6576444   21210.26437184  22990.67904908  20824.87319461
  82444.41124241  41072.52705364  56832.13587757 104995.68554514
  40771.53069546  58919.73831746  93465.9190949  103961.08659508
  79969.41629705 102154.7

In [13]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv=4)

array([ 0.75033504, -0.39167018,  0.52341024,  0.09199341])